In [15]:
import tensorflow as tf
import numpy as np
import math

In [73]:
# Helper function
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    
    output_batches = []
    for start_i in range(0, len(features), batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i: end_i], labels[start_i: end_i]]
        output_batches.append(batch)
        
    return output_batches

def encode_onehot(labels, n_class):
    onehot_labels = np.zeros((len(labels), n_class))
    for label, onehot_label in zip(labels, onehot_labels):
        onehot_label[label-1] = 1
    return onehot_labels

In [74]:
learning_rate = 0.001
n_input = 784  # MNIST data input (28x28) <= num of nodes in input layer
n_classes = 10 # MNIST total classes 0-9 digits <= num of nodes in output layer

In [75]:
# Load MNIST data sets
mnist = tf.keras.datasets.mnist

(train_features, train_labels), (test_features, test_labels) = mnist.load_data()

train_features = train_features/255.0
test_features = test_features/255.0

train_features = np.reshape(train_features, (len(train_features), n_input))
test_features = np.reshape(test_features, (len(test_features), n_input))

train_labels = encode_onehot(train_labels, n_classes)
test_l

5
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [61]:
# construct the single layer neural network
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

w1 = tf.Variable(tf.random_normal([n_input, n_classes]))
b1 = tf.Variable(tf.random_normal([n_classes]))

logits = tf.add(tf.matmul(features, w1), b1)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [62]:
# calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [64]:
batch_size = 128

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for bat_features, bat_labels in batches(batch_size, train_features, train_labels):
        print(bat_labels.shape)
        sess.run(
            optimizer, 
            feed_dict={features: bat_features, labels: bat_labels})
        
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))


(128,)


ValueError: Cannot feed value of shape (128,) for Tensor 'Placeholder_15:0', which has shape '(?, 10)'